In [1]:
# APN Hexanomial Search and Classification - Complete Program with BC Comparison, though not for F256

from collections import Counter, defaultdict
import time

# ============================================================================
# PART 1: APN SEARCH FUNCTIONS
# ============================================================================

def check_condition_C1(A, B, C, D, E, q):
    """Check if (C1) holds - if so, NOT APN"""
    if C == 0 and D == 0 and A != 0:
        if A^q * B == B^q and A^q * E == E^q:
            return True
    return False

def check_condition_C2(A, B, C, D, E, q):
    """Check if (C2) holds - if so, NOT APN"""
    if A*C*D != 0:
        if A^(q+1) == 1 and D == A*C^q and B^q == A^q*B and E^q == A^q*E:
            return True
    return False

def compute_h1(A, B, C, D, q):
    """Compute h1 polynomial"""
    return (A^(q+1) * B^(q+1) + A * B^(2*q) + A^q * B^2 + 
            B^2 * C^q * D^q + B^(q+1) * C^(q+1) + 
            B^(q+1) * D^(q+1) + B^(q+1) + B^(2*q) * C * D)

def check_condition_C6(A, C, D, q):
    """Check if (C6) holds"""
    return (A*D^q + C != 0 or 
            A^(q+1) + C^(q+1) + D^(q+1) + 1 != 0)

def check_condition_C7(A, B, C, D, q):
    """Check if (C7) holds"""
    return compute_h1(A, B, C, D, q) != 0

def is_candidate_APN(A, B, C, D, E, q):
    """Check if parameters avoid all proven non-APN conditions."""
    if A == 0:
        return False
    
    if check_condition_C1(A, B, C, D, E, q):
        return False
    if check_condition_C2(A, B, C, D, E, q):
        return False
    
    if check_condition_C6(A, C, D, q) and check_condition_C7(A, B, C, D, q):
        return False
    
    return True

def get_field_info(F, q):
    """Get minimal polynomial and generator information for the field."""
    gen = F.gen()
    min_poly = F.modulus()
    return gen, min_poly
    
def test_APN_property(A, B, C, D, E, F, q):
    """Test if the function is APN"""
    for a in F:
        if a == 0:
            continue
            
        solutions = set()
        for x in F:
            val = (A*a + a^(2*q)*E + a^q*D)*x^2
            val += (a^2*A + a^(2*q)*C + a^q*B)*x
            val += (a^2*E + a*C + a^q)*x^(2*q)
            val += (a^2*D + a*B + a^(2*q))*x^q
            
            if val == 0:
                solutions.add(x)
        
        trivial = {F(0), a}
        if not solutions.issubset(trivial):
            return False
    
    return True

def format_polynomial_for_file(A, B, C, D, E, q):
    """Format polynomial for file output"""
    terms = []
    
    if A != 0:
        terms.append(f"({A})*x^3")
    if B != 0:
        terms.append(f"({B})*x^{q+1}")
    if C != 0:
        terms.append(f"({C})*x^{2*q+1}")
    if D != 0:
        terms.append(f"({D})*x^{q+2}")
    if E != 0:
        terms.append(f"({E})*x^{2*q+2}")
    
    terms.append(f"x^{3*q}")
    
    return " + ".join(terms)

def format_for_file_with_coeffs(A, B, C, D, E, q):
    """Output format: coefficients followed by polynomial"""
    coeffs = f"{A},{B},{C},{D},{E}"
    poly = format_polynomial_for_file(A, B, C, D, E, q)
    return f"{coeffs} | {poly}"

def search_APN_hexanomials(n, max_candidates=None, max_time=None, 
                           verbose=True, output_file=None, 
                           progress_interval=1000):
    """
    Search for APN hexanomials over F_{2^n}.
    """
    q = 2^n
    F = GF(q^2, 'a')
    
    if verbose:
        print(f"Searching over F_{{2^{n}}}^2 where q^2 = {q^2}")
        print(f"Field size: {q^2}, Total param space: {(q^2)**5}")
        if max_candidates:
            print(f"Max candidates to test: {max_candidates}")
        if max_time:
            print(f"Max time: {max_time} seconds")
        if output_file:
            print(f"Output file: {output_file}")
        print()
    
    candidates_tested = 0
    candidates_filtered = 0
    apn_found = []
    start_time = time.time()
    
    file_handle = None
    if output_file:
        file_handle = open(output_file, 'w')
        gen, min_poly = get_field_info(F, q)
        file_handle.write(f"# APN Hexanomials over F_{{2^{n}}}^2 (q = {q})\n")
        file_handle.write(f"# Format: A,B,C,D,E | polynomial\n")
        file_handle.write(f"# Field: GF({q^2}, 'a')\n")
        file_handle.write(f"# Primitive element: a\n")
        file_handle.write(f"# Minimal polynomial: {min_poly}\n")
        file_handle.write(f"# (i.e., a is a root of {min_poly} = 0)\n\n")
    
    try:
        if q <= 4 and max_candidates is None:
            if verbose:
                print("Running exhaustive search...\n")
            
            for A in F:
                if max_time and (time.time() - start_time) > max_time:
                    if verbose:
                        print(f"\nTime limit reached ({max_time}s)")
                    break
                    
                for B in F:
                    for C in F:
                        for D in F:
                            for E in F:
                                if is_candidate_APN(A, B, C, D, E, q):
                                    candidates_tested += 1
                                    
                                    if test_APN_property(A, B, C, D, E, F, q):
                                        apn_found.append((A, B, C, D, E))
                                        
                                        if file_handle:
                                            output_str = format_for_file_with_coeffs(A, B, C, D, E, q)
                                            file_handle.write(f"{output_str}\n")
                                            file_handle.flush()
                                        
                                        if verbose and len(apn_found) % 400 == 0:
                                            print(f"Found {len(apn_found)} APN functions...")
                                else:
                                    candidates_filtered += 1
        else:
            if verbose:
                limit_str = f"up to {max_candidates}" if max_candidates else "unlimited"
                print(f"Running random search ({limit_str} candidates)...\n")
            
            tested_set = set()
            
            while True:
                if max_candidates and candidates_tested >= max_candidates:
                    if verbose:
                        print(f"\nCandidate limit reached ({max_candidates})")
                    break
                    
                if max_time and (time.time() - start_time) > max_time:
                    if verbose:
                        print(f"\nTime limit reached ({max_time}s)")
                    break
                
                A = F.random_element()
                B = F.random_element()
                C = F.random_element()
                D = F.random_element()
                E = F.random_element()
                
                param_tuple = (A, B, C, D, E)
                if param_tuple in tested_set:
                    continue
                tested_set.add(param_tuple)
                
                if is_candidate_APN(A, B, C, D, E, q):
                    candidates_tested += 1
                    
                    if verbose and candidates_tested % progress_interval == 0:
                        elapsed = time.time() - start_time
                        rate = candidates_tested / elapsed if elapsed > 0 else 0
                        print(f"Tested: {candidates_tested} | Found: {len(apn_found)} | "
                              f"Rate: {rate:.1f} tests/sec | Elapsed: {elapsed:.1f}s")
                    
                    if test_APN_property(A, B, C, D, E, F, q):
                        apn_found.append((A, B, C, D, E))
                        
                        if file_handle:
                            output_str = format_for_file_with_coeffs(A, B, C, D, E, q)
                            file_handle.write(f"{output_str}\n")
                            file_handle.flush()
                        
                        if verbose:
                            print(f"  >>> APN #{len(apn_found)} found!")
                else:
                    candidates_filtered += 1
    
    finally:
        if file_handle:
            file_handle.close()
    
    elapsed_time = time.time() - start_time
    if verbose:
        print("\n" + "="*70)
        print("SEARCH COMPLETE")
        print("="*70)
        print(f"Candidates tested: {candidates_tested}")
        print(f"Candidates filtered: {candidates_filtered}")
        print(f"APN functions found: {len(apn_found)}")
        print(f"Total time: {elapsed_time:.2f} seconds")
        if candidates_tested > 0:
            print(f"Average rate: {candidates_tested/elapsed_time:.1f} tests/sec")
        if output_file:
            print(f"Results written to: {output_file}")
        print()
    
    return apn_found

# ============================================================================
# PART 2: BUDAGHYAN-CARLET EQUIVALENCE CHECKING
# ============================================================================

 
def generate_bc_function(m, n, c, d, F):
    """Generate a Budaghyan-Carlet function with given parameters."""
    r = 2^m
    s = 2^n
    q = r * s
    
    coeffs = {
        3: 0,
        q + 1: 0,
        2*q + 1: 0,
        q + 2: 0,
        2*q + 2: 0,
        3*q: 1
    }
    
    bc_terms = {
        s + 1: F(1),
        r + 1: F(1),
        r*s + 1: c,
        s + r: c^r,
        s + r*s: d,
        (s + 1)*r: F(1)
    }
    
    for exp, coeff in bc_terms.items():
        if exp in coeffs:
            coeffs[exp] = coeff
    
    A = coeffs[3]
    B = coeffs[q + 1]
    C = coeffs[2*q + 1]
    D = coeffs[q + 2]
    E = coeffs[2*q + 2]
    
    return A, B, C, D, E

def check_bc_conditions(m, n, c, d, F):
    """Check if BC parameters satisfy APN conditions."""
    r = 2^m
    
    if m <= 1:
        return False, "m must be > 1 (Bluher)"
    if n % m != 0:
        return False, "n/m must be an integer (Bluher)"
    if (n // m) % 2 == 0:
        return False, "n/m must be odd (Bluher)"
    
    if d^r == d:
        return False, "d must not be in F_r"
    
    return True, "Satisfies Bluher conditions and d ∉ F_r"

def get_valid_bc_parameters(n_field):
    """Get all valid (m, n) pairs satisfying Bluher's conditions for F_{2^(2*n_field)}."""
    valid_pairs = []
    total_dim = 2 * n_field  # Correctly use the full field dimension
    
    # Iterate through possible values for m (renamed k for clarity)
    for k in range(2, total_dim):  # k > 1
        j = total_dim - k
        
        # Check if j/k is an integer and odd
        if j % k == 0 and (j // k) % 2 == 1:
            valid_pairs.append((k, j))
    
    return valid_pairs

def generate_all_bc_functions(F, q):
    """Generate ALL Budaghyan-Carlet APN functions for the given field."""
    n_field = int(log(q, 2))
    valid_pairs = get_valid_bc_parameters(n_field)
    
    if not valid_pairs:
        return []
    
    all_bc_functions = []
    
    print(f"Valid (m,n) pairs satisfying Bluher's conditions: {valid_pairs}")
    
    for m, n in valid_pairs:
        r = 2^m
        s = 2^n
        
        print(f"  Generating BC functions for m={m}, n={n} (r={r}, s={s})...")
        
        count = 0
        for c in F:
            for d in F:
                if d^r == d:
                    continue
                
                A, B, C, D, E = generate_bc_function(m, n, c, d, F)
                all_bc_functions.append((A, B, C, D, E, m, n, c, d))
                count += 1
        
        print(f"    Generated {count} BC functions for this (m,n)")
    
    print(f"Total BC functions generated: {len(all_bc_functions)}\n")
    return all_bc_functions

# ============================================================================
# PART 3: CLASSIFICATION FUNCTIONS
# ============================================================================

def parse_apn_file(filename, n):
    """Parse APN file with format: A,B,C,D,E | polynomial"""
    q = 2^n
    F = GF(q^2, 'a')
    
    functions = []
    
    print(f"Reading from {filename}...")
    
    with open(filename, 'r') as f:
        for line_num, line in enumerate(f, 1):
            line = line.strip()
            if not line or line.startswith('#'):
                continue
                
            if '|' not in line:
                continue
                
            coeffs_str, poly_str = line.split('|', 1)
            coeffs_str = coeffs_str.strip()
            
            parts = coeffs_str.split(',')
            if len(parts) != 5:
                print(f"Warning: Line {line_num} has wrong format")
                continue
            
            try:
                A = F(parts[0].strip())
                B = F(parts[1].strip())
                C = F(parts[2].strip())
                D = F(parts[3].strip())
                E = F(parts[4].strip())
                
                functions.append((A, B, C, D, E))
            except Exception as e:
                print(f"Warning: Line {line_num} parse error: {e}")
                continue
    
    print(f"Loaded {len(functions)} APN functions")
    return functions, F, q

def build_function_table(A, B, C, D, E, F, q):
    """Build complete lookup table for f(x)."""
    table = {}
    for x in F:
        fx = (x * (A*x^2 + B*x^q + C*x^(2*q)) + 
              x^2 * (D*x^q + E*x^(2*q)) + 
              x^(3*q))
        table[x] = fx
    return table

def compute_differential_uniformity(func_table, F):
    """Compute differential uniformity and spectrum."""
    max_delta = 0
    spectrum = defaultdict(int)
    
    for a in F:
        if a == 0:
            continue
        
        derivative_counts = defaultdict(int)
        for x in F:
            diff = func_table[x + a] + func_table[x]
            derivative_counts[diff] += 1
        
        for count in derivative_counts.values():
            max_delta = max(max_delta, count)
            spectrum[count] += 1
    
    spectrum_tuple = tuple(sorted(spectrum.items()))
    return max_delta, spectrum_tuple

def compute_walsh_spectrum(func_table, F, q):
    """Compute Walsh spectrum distribution."""
    walsh_values = []
    
    for a in F:
        for b in F:
            walsh_sum = 0
            for x in F:
                fx = func_table[x]
                inner = a * fx + b * x
                trace_val = inner.trace()
                trace_int = ZZ(trace_val)
                walsh_sum += (-1)^trace_int
            
            walsh_values.append(abs(walsh_sum))
    
    walsh_counter = Counter(walsh_values)
    return tuple(sorted(walsh_counter.items()))

def compute_nonlinearity(walsh_spectrum):
    """Nonlinearity from Walsh spectrum."""
    if not walsh_spectrum:
        return 0
    max_walsh = max(val for val, count in walsh_spectrum)
    return max_walsh

def compute_algebraic_degree(A, B, C, D, E, q):
    """Compute algebraic degree from exponents."""
    exponents = [3, q+1, 2*q+1, q+2, 2*q+2, 3*q]
    coeffs = [A, B, C, D, E, 1]
    
    max_weight = 0
    
    for exp, coeff in zip(exponents, coeffs):
        if coeff != 0:
            weight = bin(exp % (q**2 - 1)).count('1')
            max_weight = max(max_weight, weight)
    
    return max_weight

def compute_ddt_signature(func_table, F):
    """Compute signature from DDT structure."""
    ddt = defaultdict(lambda: defaultdict(int))
    
    for a in F:
        if a == 0:
            continue
        for x in F:
            diff = func_table[x + a] + func_table[x]
            ddt[a][diff] += 1
    
    row_patterns = []
    for a in F:
        if a == 0:
            continue
        row = tuple(sorted(ddt[a].values()))
        row_patterns.append(row)
    
    row_counter = Counter(row_patterns)
    signature = tuple(sorted(row_counter.items()))
    
    return signature

def compute_autocorrelation_spectrum(func_table, F):
    """Compute autocorrelation spectrum."""
    autocorr_values = []
    
    for a in F:
        if a == 0:
            continue
        
        corr_sum = 0
        for x in F:
            fx = func_table[x]
            fx_plus_a = func_table[x + a]
            
            product = fx * fx_plus_a
            trace_val = product.trace()
            trace_int = ZZ(trace_val)
            corr_sum += (-1)^trace_int
        
        autocorr_values.append(abs(corr_sum))
    
    autocorr_counter = Counter(autocorr_values)
    return tuple(sorted(autocorr_counter.items()))

def compute_component_invariants(func_table, F, q):
    """Analyze boolean components of the function."""
    n = int(log(q, 2))
    
    component_balances = []
    component_nonlinearities = []
    
    sample_size = min(10, q - 1)
    sampled_elements = list(F)[1:sample_size+1]
    
    for beta in sampled_elements:
        values = []
        for x in F:
            fx = func_table[x]
            val = (beta * fx).trace()
            values.append(ZZ(val))
        
        balance = abs(sum(values) - len(F)//2)
        component_balances.append(balance)
        
        walsh_vals = []
        for alpha in F:
            walsh_sum = 0
            for x in F:
                fx = func_table[x]
                inner = (beta * fx + alpha * x).trace()
                walsh_sum += (-1)^ZZ(inner)
            walsh_vals.append(abs(walsh_sum))
        
        max_walsh = max(walsh_vals)
        component_nonlinearities.append(max_walsh)
    
    return (tuple(sorted(component_balances)), tuple(sorted(component_nonlinearities)))

def compute_extended_invariants(A, B, C, D, E, F, q):
    """Compute extended set of invariants for better discrimination."""
    func_table = build_function_table(A, B, C, D, E, F, q)
    
    diff_uniform, diff_spectrum = compute_differential_uniformity(func_table, F)
    walsh_spectrum = compute_walsh_spectrum(func_table, F, q)
    nonlinearity = compute_nonlinearity(walsh_spectrum)
    is_perm = len(set(func_table.values())) == len(F)
    
    algebraic_degree = compute_algebraic_degree(A, B, C, D, E, q)
    ddt_signature = compute_ddt_signature(func_table, F)
    autocorr_spectrum = compute_autocorrelation_spectrum(func_table, F)
    component_invariants = compute_component_invariants(func_table, F, q)
    
    invariants = {
        'differential_uniformity': diff_uniform,
        'differential_spectrum': diff_spectrum,
        'walsh_spectrum': walsh_spectrum,
        'nonlinearity': nonlinearity,
        'is_permutation': is_perm,
        'algebraic_degree': algebraic_degree,
        'ddt_signature': ddt_signature,
        'autocorr_spectrum': autocorr_spectrum,
        'component_invariants': component_invariants,
    }
    
    return invariants

def create_enhanced_invariant_key(inv):
    """Create a more discriminating invariant key."""
    return (
        inv['differential_uniformity'],
        inv['differential_spectrum'],
        inv['walsh_spectrum'],
        inv['is_permutation'],
        inv['algebraic_degree'],
        inv['ddt_signature'],
        inv['autocorr_spectrum'],
        inv['component_invariants'],
    )

def select_class_representative(members, F, q):
    """Select a canonical representative from an equivalence class."""
    def score_function(coeffs):
        A, B, C, D, E, inv = coeffs
        
        non_zero = sum([A != 0, B != 0, C != 0, D != 0, E != 0])
        is_perm = inv['is_permutation']
        int_repr = tuple(c.polynomial().list() for c in [A, B, C, D, E])
        
        return (non_zero, not is_perm, int_repr)
    
    representative = min(members, key=score_function)
    return representative[:5]

def format_function_readable(A, B, C, D, E, q):
    """Format function in most readable form."""
    term_dict = {}
    
    if A != 0:
        term_dict[3] = term_dict.get(3, 0) + A
    if B != 0:
        exp = int(q+1)
        term_dict[exp] = term_dict.get(exp, 0) + B
    if C != 0:
        exp = int(2*q+1)
        term_dict[exp] = term_dict.get(exp, 0) + C
    if D != 0:
        exp = int(q+2)
        term_dict[exp] = term_dict.get(exp, 0) + D
    if E != 0:
        exp = int(2*q+2)
        term_dict[exp] = term_dict.get(exp, 0) + E
    
    exp = int(3*q)
    term_dict[exp] = term_dict.get(exp, 0) + 1
    
    terms = [(coeff, exp) for exp, coeff in term_dict.items() if coeff != 0]
    terms.sort(key=lambda x: x[1])
    
    num_terms = len(terms)
    type_names = {1: "Monomial", 2: "Binomial", 3: "Trinomial", 
                  4: "Quadrinomial", 5: "Pentanomial", 6: "Hexanomial"}
    func_type = type_names.get(num_terms, "Hexanomial")
    
    poly_parts = []
    for coeff, exp in terms:
        if coeff == 1:
            poly_parts.append(f"x^{exp}")
        else:
            poly_parts.append(f"({coeff})*x^{exp}")
    
    poly_str = " + ".join(poly_parts)
    
    return func_type, poly_str
def compare_with_bc_functions(found_functions, F, q, output_file):
    """
    Generates all BC functions, classifies them by invariants, and compares
    them against the classes of the found APN functions.
    """
    print(f"Generating all Budaghyan-Carlet functions for F_{q^2}...")
    
    # Generate all possible BC functions for this field
    all_bc_funcs = generate_all_bc_functions(F, q)
    
    if not all_bc_funcs:
        print("No valid (m,n) parameters for BC functions in this field.")
        if output_file:
            with open(output_file, 'w') as f:
                f.write(f"# No valid Budaghyan-Carlet functions for F_{q^2}.\\n")
        return None

    print(f"\\nComputing invariants for {len(all_bc_funcs)} BC functions...")
    
    start_time = time.time()
    bc_invariant_classes = defaultdict(list)
    
    for idx, (A, B, C, D, E, m, n, c, d) in enumerate(all_bc_funcs):
        if (idx + 1) % 5000 == 0:
            elapsed = time.time() - start_time
            rate = (idx + 1) / elapsed if elapsed > 0 else 0
            print(f"  Processed {idx + 1}/{len(all_bc_funcs)} ({rate:.1f} funcs/sec)")

        # Compute invariants and classify the BC function
        inv = compute_extended_invariants(A, B, C, D, E, F, q)
        inv_key = create_enhanced_invariant_key(inv)
        
        # Store detailed info about the BC function
        bc_info = {
            'coeffs': (A, B, C, D, E),
            'm': m,
            'n': n,
            'c': c,
            'd': d,
            'invariants': inv
        }
        bc_invariant_classes[inv_key].append(bc_info)

    elapsed = time.time() - start_time
    print(f"\\nBC Invariant computation complete in {elapsed:.1f} seconds.")
    print(f"Found {len(bc_invariant_classes)} distinct BC invariant classes.\\n")

    # Write results to the output file
    if output_file:
        print(f"Writing BC classification to {output_file}...")
        with open(output_file, 'w') as f:
            f.write(f"# Budaghyan-Carlet (BC) Function Classification for F_{q^2}\\n")
            f.write(f"# Total BC functions generated: {len(all_bc_funcs)}\\n")
            f.write(f"# Distinct invariant classes: {len(bc_invariant_classes)}\\n\\n")

            sorted_bc_classes = sorted(bc_invariant_classes.items(), key=lambda x: len(x[1]), reverse=True)

            for class_num, (inv_key, members) in enumerate(sorted_bc_classes, 1):
                # Select a representative from the BC members
                rep_coeffs = members[0]['coeffs']
                rep_A, rep_B, rep_C, rep_D, rep_E = rep_coeffs
                func_type, poly_str = format_function_readable(rep_A, rep_B, rep_C, rep_D, rep_E, q)

                f.write(f"{'=' * 70}\\n")
                f.write(f"BC CLASS {class_num}: {len(members)} functions\\n")
                f.write(f"{'=' * 70}\\n")
                f.write(f"REPRESENTATIVE ({func_type}):\\n")
                f.write(f"  f(x) = {poly_str}\\n\\n")
                
                sample_inv = members[0]['invariants']
                f.write(f"INVARIANTS:\\n")
                f.write(f"  Differential uniformity: {sample_inv['differential_uniformity']}\\n")
                f.write(f"  Nonlinearity: {sample_inv['nonlinearity']}\\n")
                f.write(f"  Algebraic degree: {sample_inv['algebraic_degree']}\\n\\n")

                f.write(f"EXAMPLE MEMBERS (first 10):\\n")
                for member_info in members[:10]:
                    m, n, c, d = member_info['m'], member_info['n'], member_info['c'], member_info['d']
                    f.write(f"  - (m={m}, n={n}, c={c}, d={d})\\n")
                if len(members) > 10:
                    f.write(f"  ... and {len(members) - 10} more\\n")
                f.write("\\n")

    # Return a dictionary structured for the main classification function
    return {
        'total_bc_functions': len(all_bc_funcs),
        'bc_invariant_classes': bc_invariant_classes
    }
    
def classify_with_enhanced_invariants(functions, F, q, output_file, bc_results=None):
    """Classify using enhanced invariants with BC information if available."""
    print(f"\nComputing enhanced invariants for {len(functions)} functions...")
    print("This includes: algebraic degree, DDT signatures, autocorrelation,")
    print("and component function analysis...\n")
    
    start_time = time.time()
    
    invariant_classes = defaultdict(list)
    
    for idx, (A, B, C, D, E) in enumerate(functions):
        if (idx + 1) % 20 == 0:
            elapsed = time.time() - start_time
            rate = (idx + 1) / elapsed
            print(f"Processed {idx + 1}/{len(functions)} ({rate:.1f} funcs/sec)")
        
        inv = compute_extended_invariants(A, B, C, D, E, F, q)
        inv_key = create_enhanced_invariant_key(inv)
        invariant_classes[inv_key].append((A, B, C, D, E, inv))
    
    elapsed = time.time() - start_time
    print(f"\nInvariant computation completed in {elapsed:.1f} seconds")
    print(f"Found {len(invariant_classes)} distinct invariant classes")
    
    print(f"\nDETAILED BREAKDOWN:")
    print(f"  Total functions: {len(functions)}")
    print(f"  Distinct classes: {len(invariant_classes)}")
    
    if len(invariant_classes) == 1:
        print(f"\n  WARNING: All functions have identical invariants!")
        print(f"  This suggests either:")
        print(f"    1. They are all CCZ-equivalent (rare but possible)")
        print(f"    2. They differ in ways these invariants don't capture")
        print(f"    3. There's a bug in the search/classification")
        
        print(f"\n  Performing sample pairwise checks...")
        sample_functions = list(invariant_classes.values())[0][:5]
        for i in range(min(3, len(sample_functions))):
            f1 = sample_functions[i][:5]
            f2 = sample_functions[i+1][:5] if i+1 < len(sample_functions) else sample_functions[0][:5]
            print(f"    Function {i+1}: {f1}")
            print(f"    Function {i+2 if i+1 < len(sample_functions) else 1}: {f2}")
            print(f"    Coefficients differ: {f1 != f2}")
    
    bc_class_info = {}
    if bc_results:
        print(f"\nCross-referencing with BC functions...")
        bc_invariants = bc_results.get('bc_invariant_classes', {})
        
        for inv_key, members in invariant_classes.items():
            if inv_key in bc_invariants:
                bc_class_info[inv_key] = {
                    'is_bc_class': True,
                    'num_bc_functions': len(bc_invariants[inv_key]),
                    'bc_examples': bc_invariants[inv_key][:3]
                }
            else:
                bc_class_info[inv_key] = {'is_bc_class': False}
    
    print(f"\nWriting results to {output_file}...\n")
    
    with open(output_file, 'w') as f:
        gen, min_poly = get_field_info(F, q)
        f.write(f"# Enhanced CCZ-Invariant Classification\n")
        f.write(f"# Total functions: {len(functions)}\n")
        f.write(f"# Distinct invariant classes: {len(invariant_classes)}\n")
        f.write(f"# Field: F_{{2^{int(log(q,2))}}}^2 (q = {q})\n")
        f.write(f"# Invariants used: differential uniformity, differential spectrum,\n")
        f.write(f"#   Walsh spectrum, permutation property, algebraic degree,\n")
        f.write(f"#   DDT signature, autocorrelation spectrum, component invariants\n")
        
        if bc_results:
            f.write(f"#\n# BC COMPARISON:\n")
            bc_classes_count = sum(1 for info in bc_class_info.values() if info['is_bc_class'])
            f.write(f"#   Classes containing BC functions: {bc_classes_count}/{len(invariant_classes)}\n")
            f.write(f"#   Classes with NO BC functions: {len(invariant_classes) - bc_classes_count}\n")
            f.write(f"\n")
        
        sorted_classes = sorted(invariant_classes.items(), 
                               key=lambda x: len(x[1]), reverse=True)
        
        for class_num, (inv_key, members) in enumerate(sorted_classes, 1):
            rep_A, rep_B, rep_C, rep_D, rep_E = select_class_representative(members, F, q)
            func_type, poly_str = format_function_readable(rep_A, rep_B, rep_C, rep_D, rep_E, q)
            
            f.write(f"{'=' * 70}\n")
            f.write(f"CLASS {class_num}: {len(members)} functions")
            
            if inv_key in bc_class_info:
                if bc_class_info[inv_key]['is_bc_class']:
                    num_bc = bc_class_info[inv_key]['num_bc_functions']
                    f.write(f" *** CONTAINS BUDAGHYAN-CARLET FUNCTIONS ***\n")
                    f.write(f"    (This CCZ-class contains {num_bc} BC functions)\n")
                else:
                    f.write(f" *** NO BC FUNCTIONS - POTENTIALLY NEW! ***\n")
            else:
                f.write(f"\n")
            
            f.write(f"{'=' * 70}\n")
            
            f.write(f"\nREPRESENTATIVE ({func_type}):\n")
            f.write(f"  Coefficients: A={rep_A}, B={rep_B}, C={rep_C}, D={rep_D}, E={rep_E}\n")
            f.write(f"  f(x) = {poly_str}\n\n")
            
            sample_inv = members[0][5]
            f.write(f"INVARIANTS:\n")
            f.write(f"  Differential uniformity: {sample_inv['differential_uniformity']}\n")
            f.write(f"  Is permutation: {sample_inv['is_permutation']}\n")
            f.write(f"  Nonlinearity: {sample_inv['nonlinearity']}\n")
            f.write(f"  Algebraic degree: {sample_inv['algebraic_degree']}\n")
            f.write(f"  Differential spectrum: {sample_inv['differential_spectrum']}\n")
            f.write(f"  DDT signature (first 3): {str(sample_inv['ddt_signature'][:3])}\n")
            f.write(f"  Autocorr spectrum (first 3): {str(sample_inv['autocorr_spectrum'][:3])}\n")
            f.write(f"  Walsh spectrum (first 5): {str(sample_inv['walsh_spectrum'][:5])}\n")
            
            if inv_key in bc_class_info and bc_class_info[inv_key]['is_bc_class']:
                f.write(f"\nBC FUNCTION EXAMPLES IN THIS CLASS:\n")
                for bc_ex in bc_class_info[inv_key]['bc_examples']:
                    A, B, C, D, E = bc_ex['coeffs']
                    m, n, c, d = bc_ex['m'], bc_ex['n'], bc_ex['c'], bc_ex['d']
                    f.write(f"  BC function: m={m}, n={n}, c={c}, d={d}\n")
                    f.write(f"    Coefficients: {A},{B},{C},{D},{E}\n")
            
            f.write(f"\nALL FUNCTIONS IN CLASS (first 20):\n")
            for i, (A, B, C, D, E, inv) in enumerate(members[:20]):
                ftype, pstr = format_function_readable(A, B, C, D, E, q)
                f.write(f"  {i+1}. {A},{B},{C},{D},{E}\n")
            
            if len(members) > 20:
                f.write(f"  ... and {len(members) - 20} more\n")
            
            f.write("\n")
            f.flush()
    
    print(f"\nClassification complete! Results written to {output_file}")
    return invariant_classes

# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    # Search over F_4
    print("=" * 70)
    print("SEARCH 1: F_4 (exhaustive)")
    print("=" * 70)
    apn_F4 = search_APN_hexanomials(n=1, output_file="apn_hexanomials_F4.txt")
    
    # BC comparison for F_4
    print("\n" + "=" * 70)
    print("BC EQUIVALENCE CHECK: F_4")
    print("=" * 70)
    functions_F4, F4, q4 = parse_apn_file("apn_hexanomials_F4.txt", n=1)
    bc_results_F4 = compare_with_bc_functions(
        functions_F4, F4, q4, "apn_bc_comparison_F4.txt"
    )
    
    # Classify F_4 with BC info
    print("\n" + "=" * 70)
    print("CLASSIFYING F_4 APN FUNCTIONS")
    print("=" * 70)
    classes_F4 = classify_with_enhanced_invariants(
        functions_F4, F4, q4, "apn_CCZclassified_F4.txt", bc_results_F4
    )
    
    # Search over F_16
    print("\n" + "=" * 70)
    print("SEARCH 2: F_16")
    print("=" * 70)
    apn_F16 = search_APN_hexanomials(n=2, output_file="apn_hexanomials_F16.txt")
    
    # BC comparison for F_16
    print("\n" + "=" * 70)
    print("BC EQUIVALENCE CHECK: F_16")
    print("=" * 70)
    functions_F16, F16, q16 = parse_apn_file("apn_hexanomials_F16.txt", n=2)
    bc_results_F16 = compare_with_bc_functions(
        functions_F16, F16, q16, "apn_bc_comparison_F16.txt"
    )
    
    # Classify F_16 with BC info
    print("\n" + "=" * 70)
    print("CLASSIFYING F_16 APN FUNCTIONS")
    print("=" * 70)
    classes_F16 = classify_with_enhanced_invariants(
        functions_F16, F16, q16, "apn_CCZclassified_F16.txt", bc_results_F16
    )

    # Search over F_64 with configurable limits
    print("\n" + "=" * 70)
    print("SEARCH 3: F_64 (limited random search)")
    print("=" * 70)
    
    MAX_CANDIDATES_F64 = 60000
    MAX_TIME_F64 = 6000
    PROGRESS_INTERVAL_F64 = 3000
    
    apn_F64 = search_APN_hexanomials(
        n=3, 
        max_candidates=MAX_CANDIDATES_F64,
        max_time=MAX_TIME_F64,
        output_file="apn_hexanomials_F64.txt",
        progress_interval=PROGRESS_INTERVAL_F64
    )
    
    # Only classify if we found some functions
    if len(apn_F64) > 0:
        # BC comparison for F_64
        print("\n" + "=" * 70)
        print("BC EQUIVALENCE CHECK: F_64")
        print("=" * 70)
        functions_F64, F64, q64 = parse_apn_file("apn_hexanomials_F64.txt", n=3)
        bc_results_F64 = compare_with_bc_functions(
            functions_F64, F64, q64, "apn_bc_comparison_F64.txt"
        )
        
        # Classify F_64 with BC info
        print("\n" + "=" * 70)
        print("CLASSIFYING F_64 APN FUNCTIONS")
        print("=" * 70)
        classes_F64 = classify_with_enhanced_invariants(
            functions_F64, F64, q64, "apn_CCZclassified_F64.txt", bc_results_F64
        )
    else:
        print("\nNo APN functions found for F_64 in this search.")
        print("Try increasing MAX_CANDIDATES_F64 or MAX_TIME_F64")
        
    # Search over F_256 with configurable limits
    print("\n" + "=" * 70)
    print("SEARCH: F_256 (limited random search)")
    print("=" * 70)
    
    MAX_CANDIDATES_F256 = 120000
    MAX_TIME_F256 = 6000
    PROGRESS_INTERVAL_F256 = 3000
    
    apn_F256 = search_APN_hexanomials(
        n=4, 
        max_candidates=MAX_CANDIDATES_F256,
        max_time=MAX_TIME_F256,
        output_file="apn_hexanomials_F256.txt",
        progress_interval=PROGRESS_INTERVAL_F256
    )
    
    # Only classify if we found some functions
    if len(apn_F256) > 0:
        # First, parse the file to load the functions that were found
        functions_F256, F256, q256 = parse_apn_file("apn_hexanomials_F256.txt", n=4)
        
        # --- BC COMPARISON AND CLASSIFICATION SKIPPED TO SAVE TIME ---
        
        # Now, classify the functions WITHOUT BC info
        print("\n" + "=" * 70)
        print("CLASSIFYING F_256 APN FUNCTIONS (SKIPPED DUE TO COMPUTATIONAL COST)")
        print("=" * 70)
        # The following lines are commented out to prevent the multi-day computation:
        # classes_F256 = classify_with_enhanced_invariants(
        #     functions_F256, F256, q256, "apn_CCZclassified_F256.txt", bc_results=None
        # )
        print("\nF_256 classification skipped. APN coefficients saved to apn_hexanomials_F256.txt.")
    else:
        print("\nNo APN functions found for F_256 in this search.")
        print("Try increasing MAX_CANDIDATES_F256 or MAX_TIME_F256")

SEARCH 1: F_4 (exhaustive)
Searching over F_{2^1}^2 where q^2 = 4
Field size: 4, Total param space: 1024
Output file: apn_hexanomials_F4.txt

Running exhaustive search...


SEARCH COMPLETE
Candidates tested: 504
Candidates filtered: 520
APN functions found: 390
Total time: 0.15 seconds
Average rate: 3327.7 tests/sec
Results written to: apn_hexanomials_F4.txt


BC EQUIVALENCE CHECK: F_4
Reading from apn_hexanomials_F4.txt...
Loaded 390 APN functions
Generating all Budaghyan-Carlet functions for F_4...
No valid (m,n) parameters for BC functions in this field.

CLASSIFYING F_4 APN FUNCTIONS

Computing enhanced invariants for 390 functions...
This includes: algebraic degree, DDT signatures, autocorrelation,
and component function analysis...

Processed 20/390 (171.0 funcs/sec)
Processed 40/390 (205.8 funcs/sec)
Processed 60/390 (221.2 funcs/sec)
Processed 80/390 (229.6 funcs/sec)
Processed 100/390 (233.5 funcs/sec)
Processed 120/390 (237.5 funcs/sec)
Processed 140/390 (239.8 funcs/sec)
Pro